In [ ]:
import spike_data_augmentation
import spike_data_augmentation.transforms as transforms
import spike_data_augmentation.representations as representations
from joblib import Parallel, delayed
from multiprocessing import Process, Manager
from math import sqrt
import numpy as np
import time
import os
import ipdb

In [ ]:
surface_dimensions = (7,7)
representation = representations.Timesurface(surface_dimensions=surface_dimensions, tau=5e3, decay='lin', merge_polarities=True)

testset = spike_data_augmentation.datasets.NMNIST(save_to='./data', train=False, download=True, representation=representation)

In [ ]:
from multiprocessing import Pool
start_time = time.time()

def process_recording(recording):
    surf = representation(recording, testset.sensor_size, testset.ordering)
    surf = surf.reshape(-1, surface_dimensions[0], surface_dimensions[1])
    surfaces_list_proxy.append(surf)

if __name__ == '__main__':
    with Manager() as manager:
        surfaces_list_proxy = manager.list()
        pool = Pool()
        pool.map(process_recording, testset.data)
        all_surfaces = list(surfaces_list_proxy)
        print("--- %s seconds ---" % (time.time() - start_time))